In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
#load data and resize to 224,224 by class depending on folder name
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train = train_datagen.flow_from_directory(os.getcwd()+'/data/train/',
                                                    target_size = (224, 224),
                                                    batch_size = 8,
                                                    class_mode = 'categorical')

valid = train_datagen.flow_from_directory(os.getcwd()+'/data/valid/',
                                                    target_size = (224, 224),
                                                    batch_size = 8,
                                                    class_mode = 'categorical')

test = test_datagen.flow_from_directory(os.getcwd()+'/data/test/',
                                                    target_size = (224, 224),
                                                    batch_size = 8,
                                                    class_mode = 'categorical')

In [ ]:
#load model
vgg16=tf.keras.applications.vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(224, 224, 3))

In [ ]:
vgg16.summary()

In [ ]:
# freeze layers
# for l in vgg16.layers:
#   l.trainable = False

In [ ]:
#create model from frozen layer except last layer because it works on 1000 classes and instead make it classify 2 classes
model=tf.keras.Sequential()

# for l in vgg16.layers[:-1]:
#   model.add(l)
model.add(vgg16)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
model.summary()

In [ ]:
with tf.device('/gpu:0'):
  hist=model.fit(train,validation_data=valid,epochs=10,batch_size=8,verbose=1)